# Build the Classifier
This notebook is dedicated to creating classifiers and run classification analyses of interest on neuroimaging data.

Can we accurately classify:
- adults vs. children
- condition within adults
- condition within children

In [5]:
from pandas import DataFrame, Series, read_csv

# Study specific variables
study_home = '/home/camachocm2/Analysis/KidVid_MVPA'
standard_mask = study_home + '/template/MNI152_T1_2mm_brain_mask_KV.nii.gz'
template = study_home + '/template/MNI152_T1_1mm_brain.nii.gz'
sub_data_file = study_home + '/doc/subjectinfo.csv'
preproc_dir = study_home + '/analysis/preproc/betas'
output_dir = study_home + '/analysis/classifier'

condition_data = read_csv(study_home + '/doc/conditionslist.csv')
subject_info = read_csv(sub_data_file)

In [8]:
## Create a conditions list for the feature set
condition_labels = condition_data['labels'].tolist()
subjects_list = subject_info['subjID'].tolist()
age_group_list = subject_info['group'].tolist()
ages_mos_list = subject_info['age_mos'].tolist()
mapdb_temploss_list = subject_info['MAPDB_temploss_std'].tolist()
cbq_angfrust_list = subject_info['CBQ_Anger_Frustration_std'].tolist()


conditions = condition_data
conditions['subject'] = Series(subjects_list[0], index=conditions.index)
conditions['ageGroup'] = Series(age_group_list[0], index=conditions.index)
conditions['age'] = Series(ages_mos_list[0], index=conditions.index)
conditions['MAPDB'] = Series(mapdb_temploss_list[0], index=conditions.index)
conditions['CBQ'] = Series(cbq_angfrust_list[0], index=conditions.index)

for a in range(1,len(subjects_list)):
    temp=DataFrame()
    temp['labels'] = Series(condition_labels)
    temp['subject'] = Series(subjects_list[a], index=temp.index)
    temp['ageGroup'] = Series(age_group_list[a], index=temp.index)
    temp['age'] = Series(ages_mos_list[a], index=temp.index)
    temp['MAPDB'] = Series(mapdb_temploss_list[a], index=temp.index)
    temp['CBQ'] = Series(cbq_angfrust_list[a], index=temp.index)
    conditions = conditions.append(temp, ignore_index=True)

#conditions.to_csv(output_dir + '/featureset_key.csv')
conditions.describe()

,age,MAPDB,CBQ
count,1224.000000,7.200000e+02,7.200000e+02
mean,186.007843,-3.333353e-12,-3.333294e-12
std,118.706330,1.000695e+00,1.000695e+00
min,58.000000,-1.568304e+00,-2.255473e+00
25%,84.500000,-9.273081e-01,-6.841020e-01
50%,122.100000,-9.401280e-02,-4.126833e-02
75%,288.000000,6.110832e-01,7.444172e-01
max,528.000000,2.405873e+00,1.887233e+00


In [3]:
## Temporally concatenate all the parameter estimates from preproc to create a feature set
from glob import glob
from nipype.interfaces.fsl.utils import Merge
files = glob(preproc_dir + '/*/betas.nii.gz')
files = sorted(files)

bold_feature_data = output_dir + '/featureset.nii.gz'

merge = Merge()
merge.inputs.in_files = files
merge.inputs.dimension = 't'
merge.inputs.merged_file = bold_feature_data
#merge.run()

In [15]:
# determine which analysis to run
analysis = 'age_neg'

if analysis == 'all_conditions':
    mask = conditions['labels'].isin(['negative','positive','neutral'])
    labels = conditions['labels']
elif analysis == 'adults':
    mask = conditions['ageGroup'].isin(['adult'])
    labels = conditions['labels']
elif analysis == 'children':
    mask = conditions['ageGroup'].isin(['child'])
    labels = conditions['labels']
elif analysis == 'allConds_predAge':
    mask = conditions['labels'].isin(['negative','positive','neutral'])
    labels = conditions['ageGroup']
elif analysis == 'negative':
    mask = conditions['labels'].isin(['negative'])
    labels = conditions['ageGroup']
elif analysis == 'positive':
    mask = conditions['labels'].isin(['positive'])
    labels = conditions['ageGroup']
elif analysis == 'neutral':
    mask = conditions['labels'].isin(['neutral'])
    labels = conditions['ageGroup']
elif analysis == 'age_neg':
    mask = (conditions['ageGroup']=='child') & (conditions['labels']=='negative')
    labels = conditions['age']
elif analysis == 'age_pos':
    mask = (conditions['ageGroup']=='child') & (conditions['labels']=='positive')
    labels = conditions['age']
elif analysis == 'age_neu':
    mask = (conditions['ageGroup']=='child') & (conditions['labels']=='neutral')
    labels = conditions['age']

conditions[mask].describe()
#results_file = open(output_dir + '/results_' + analysis + '.txt','w')

,age,MAPDB,CBQ
count,240.000000,2.400000e+02,2.400000e+02
mean,93.013333,-3.333356e-12,-3.333330e-12
std,21.279644,1.002090e+00,1.002090e+00
min,58.000000,-1.568304e+00,-2.255473e+00
25%,73.400000,-9.273081e-01,-6.841020e-01
50%,91.000000,-9.401280e-02,-4.126833e-02
75%,114.000000,6.110832e-01,7.444172e-01
max,131.400000,2.405873e+00,1.887233e+00


## Perform the actual support vector classification

In [5]:
# Perform the support vector classification
from nilearn.input_data import NiftiMasker
from sklearn.svm import SVC
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import Pipeline

# Set up the support vector classifier
svc = SVC(kernel='linear')
masker = NiftiMasker(mask_img=standard_mask,standardize=True, 
                     memory='nilearn_cache', memory_level=1)
feature_selection = SelectPercentile(f_classif, percentile=5)
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

# Run the classifier
X = masker.fit_transform(bold_feature_data)
X = X[mask]
maskedlabels=labels[mask]
anova_svc.fit(X, maskedlabels)
y_pred = anova_svc.predict(X)

# Obtain prediction values via cross validation
from sklearn.cross_validation import LeaveOneLabelOut, cross_val_score

cv = LeaveOneLabelOut(conditions['subject'][mask])
cv_scores = cross_val_score(anova_svc, X, maskedlabels, cv=cv)
classification_accuracy = cv_scores.mean()


print("Classification accuracy: %.4f / Chance level: %f" % 
      (classification_accuracy, 1. / len(labels.unique())))

results_file.write("Classification accuracy: %.4f / Chance level: %f \n" % (classification_accuracy, 1. / len(labels.unique())))

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Classification accuracy: 0.7132 / Chance level: 0.500000


## Perform permutation testing to get a p-value for the classifier

In [ ]:
from sklearn.model_selection import permutation_test_score
import matplotlib.pyplot as plt
from numpy import savetxt

# Perform permutation testing to get a p-value
score, permutation_scores, pvalue = permutation_test_score(svc, X, y_pred, scoring="accuracy", 
                                                           cv=cv, n_permutations=500, n_jobs=30)
savetxt(output_dir + '/permutation_scores_' + analysis + '.txt', permutation_scores)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

plt.hist(permutation_scores, 20, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)
plt.plot(2 * [1. / len(labels.unique())], ylim, '--k', linewidth=3, label='Luck')

plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.savefig(output_dir + '/permutation_plot_' + analysis + '.png', transparent=True)
plt.show()

results_file.write("Classification score %s (pvalue : %s)" % (score, pvalue))
results_file.close()


## Visualize the SVM weights
Save the SVM weights as niftis to ease in visualization.

In [ ]:
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)
weight_img.to_filename(output_dir + '/svmweights_'+ analysis +'.nii.gz')

In [ ]:
#Shelve the results
import shelve

filename = output_dir + '/' + analysis + '_shelved.out'
my_shelf = shelve.open(filename,'n') # 'n' for new

for key in dir():
    try:
        my_shelf[key] = globals()[key]
    except TypeError:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
my_shelf.close()